In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)
running_order.head()

(5086, 20)

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,3,MS Parma 1,MS Parma,FIN,1,6.0,anastasia rudnaya,1,Anastasia Rudnaya,4.0,1.928,0.096,6.891132,1.875490,2.002510,0.063510,0.100000,1.928,0.096000
1,1,3,MS Parma 1,MS Parma,FIN,2,5.7,anni heikkonen,2,Anni Heikkonen,6.0,2.046,0.098,7.056842,1.808388,2.007626,0.099619,0.100000,2.046,0.098000
2,2,3,MS Parma 1,MS Parma,FIN,3,7.3,tiia istolahti,3,Tiia Istolahti,1.0,1.916,0.109,6.849927,1.851113,2.107522,0.128204,0.128204,1.916,0.128204
3,3,3,MS Parma 1,MS Parma,FIN,4,7.9,maija sianoja,4,Maija Sianoja,7.0,1.950,0.086,6.407751,1.869919,2.000480,0.065281,0.100000,1.950,0.086000
4,4,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,1,6.0,fiona bunn,1,Fiona Bunn,1.0,2.110,0.172,6.743671,1.863840,2.033659,0.084909,0.100000,2.110,0.100000


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,3,MS Parma 1,1,6.0,Anastasia Rudnaya,1.928,0.096000
1,3,MS Parma 1,2,5.7,Anni Heikkonen,2.046,0.098000
2,3,MS Parma 1,3,7.3,Tiia Istolahti,1.916,0.128204
3,3,MS Parma 1,4,7.9,Maija Sianoja,1.950,0.086000
4,8,Södertälje Nykvarn Orientering 1,1,6.0,Fiona Bunn,2.110,0.100000


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                    \
leg                                     1                                 2   
team_id                                                                       
3                              MS Parma 1                        MS Parma 1   
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
26                                 Lynx 1                            Lynx 1   
27                        Lillomarka OL 1                   Lillomarka OL 1   
30                     Suunta Jyväskylä 1                Suunta Jyväskylä 1   

                                                                             \
leg                                     3                                 4   
team_id                                                                       
3                              MS Parma 1                        MS Parma 1   
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
26                                 Lynx 1                            Lynx 1   
27                        Lillomarka OL 1                   Lillomarka OL 1   
30                     Suunta Jyväskylä 1                Suunta Jyväskylä 1   

        dist                               name                      \
leg        1    2    3    4                   1                   2   
team_id                                                               
3        6.0  5.7  7.3  7.9   Anastasia Rudnaya      Anni Heikkonen   
8        6.0  5.7  7.3  7.9          Fiona Bunn         Joana Wälti   
26       6.0  5.7  7.3  7.9       Sara Karvonen     Sanni Norrbacka   
27       6.0  5.7  7.3  7.9  Kristina Yakubkina  Maria Rörvik Haver   
30       6.0  5.7  7.3  7.9   Veera Klemettinen    Elisa Viholainen   

                                                          log_mean         \
leg                       3                             4        1      2   
team_id                                                                     
3            Tiia Istolahti                 Maija Sianoja    1.928  2.046   
8            Anna Kindlundh  Jeanette Jönsson Hellstadius    2.110  2.102   
26       Karoliina Sundberg                 Aino Mänkärlä    2.001  2.051   
27            Anina Brunner              Charlotte Watson    2.023  2.286   
30       Barbora Chaloupska             Kaisa Klemettinen    2.064  2.061   

                      log_std                             
leg          3      4       1         2         3      4  
team_id                                                   
3        1.916  1.950   0.096  0.098000  0.128204  0.086  
8        2.077  2.041   0.100  0.122141  0.120000  0.134  
26       1.982  1.980   0.090  0.128000  0.092000  0.105  
27       1.985  1.999   0.105  0.151490  0.140583  0.098  
30       1.984  2.050   0.106  0.122000  0.140614  0.138

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1278, 20)

(1257, 20)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
3,34.216007,74.780571,119.681929,172.443933,49.960080,97.778624,154.012746,212.069989,41.374274,85.704413,...,38.695809,46.775790,49.960080,53.182904,63.962235,65.651483,41.374274,44.129056,49.552665,55.521563
8,40.670099,83.095628,136.494574,192.166505,60.345639,112.502614,176.776294,244.314657,49.621422,96.645159,...,45.893713,46.756444,60.345639,59.413086,73.668861,79.175057,49.621422,46.716714,58.219842,60.844695
26,37.103008,76.319282,126.501598,180.292753,52.906659,103.614140,159.414269,220.781366,44.398649,88.972249,...,44.139844,46.394402,52.906659,56.647135,63.501614,70.011868,44.398649,44.343902,52.984368,57.178285
27,36.849211,84.667304,133.180032,189.244841,55.346112,123.477743,181.995556,242.733413,45.336630,101.682880,...,40.373265,48.123768,55.346112,75.228039,70.064140,70.352501,45.336630,55.962364,53.247525,58.229758
30,38.410719,78.873966,126.895362,183.104290,58.098192,108.042008,167.444704,236.114483,47.319955,92.386689,...,40.201314,46.836919,58.098192,56.742651,69.959342,80.733191,47.319955,44.744273,53.019474,61.408171


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,name_3,name_4,log_mean_1,log_mean_2,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4
team_id,,,,,,,,,,,,,,,,,,,,
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,Tiia Istolahti,Maija Sianoja,1.93,2.05,1.92,1.95,0.10,0.10,0.13,0.09
8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,Joana Wälti,Anna Kindlundh,Jeanette Jönsson Hellstadius,2.11,2.10,2.08,2.04,0.10,0.12,0.12,0.13
26,Lynx 1,Lynx 1,Lynx 1,Lynx 1,6.0,5.7,7.3,7.9,Sara Karvonen,Sanni Norrbacka,Karoliina Sundberg,Aino Mänkärlä,2.00,2.05,1.98,1.98,0.09,0.13,0.09,0.10
27,Lillomarka OL 1,Lillomarka OL 1,Lillomarka OL 1,Lillomarka OL 1,6.0,5.7,7.3,7.9,Kristina Yakubkina,Maria Rörvik Haver,Anina Brunner,Charlotte Watson,2.02,2.29,1.98,2.00,0.10,0.15,0.14,0.10
30,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,6.0,5.7,7.3,7.9,Veera Klemettinen,Elisa Viholainen,Barbora Chaloupska,Kaisa Klemettinen,2.06,2.06,1.98,2.05,0.11,0.12,0.14,0.14


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,38.695809,46.775790,49.960080,53.182904,63.962235,65.651483,41.374274,44.129056,49.552665,55.521563
8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,Joana Wälti,...,45.893713,46.756444,60.345639,59.413086,73.668861,79.175057,49.621422,46.716714,58.219842,60.844695
26,Lynx 1,Lynx 1,Lynx 1,Lynx 1,6.0,5.7,7.3,7.9,Sara Karvonen,Sanni Norrbacka,...,44.139844,46.394402,52.906659,56.647135,63.501614,70.011868,44.398649,44.343902,52.984368,57.178285
27,Lillomarka OL 1,Lillomarka OL 1,Lillomarka OL 1,Lillomarka OL 1,6.0,5.7,7.3,7.9,Kristina Yakubkina,Maria Rörvik Haver,...,40.373265,48.123768,55.346112,75.228039,70.064140,70.352501,45.336630,55.962364,53.247525,58.229758
30,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,Suunta Jyväskylä 1,6.0,5.7,7.3,7.9,Veera Klemettinen,Elisa Viholainen,...,40.201314,46.836919,58.098192,56.742651,69.959342,80.733191,47.319955,44.744273,53.019474,61.408171


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist               name  \
team_id leg                                                              
3       1                          MS Parma 1   6.0  Anastasia Rudnaya   
        2                          MS Parma 1   5.7     Anni Heikkonen   
        3                          MS Parma 1   7.3     Tiia Istolahti   
        4                          MS Parma 1   7.9      Maija Sianoja   
8       1    Södertälje Nykvarn Orientering 1   6.0         Fiona Bunn   
        2    Södertälje Nykvarn Orientering 1   5.7        Joana Wälti   
        3    Södertälje Nykvarn Orientering 1   7.3     Anna Kindlundh   

             log_mean   log_std  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
3       1       1.928  0.096000    34.216007   49.960080   41.374274   
        2       2.046  0.098000    74.780571   97.778624   85.704413   
        3       1.916  0.128204   119.681929  154.012746  135.525621   
        4       1.950  0.086000   172.443933  212.069989  191.191155   
8       1       2.110  0.100000    40.670099   60.345639   49.621422   
        2       2.102  0.122141    83.095628  112.502614   96.645159   
        3       2.077  0.120000   136.494574  176.776294  155.377390   

               fin_mean  fin_sum_log_mean  fin_sum_log_std  mass_start  \
team_id leg                                                              
3       1     41.510883          3.721391         0.095551         0.0   
        2     85.800359          4.449700         0.068194         0.0   
        3    135.807935          4.909137         0.064870         0.0   
        4    191.517743          5.253597         0.052600         0.0   
8       1     49.856755          3.904150         0.100094         0.0   
        2     96.937940          4.571040         0.077861         0.0   
        3    155.617586          5.045169         0.066844         0.0   

             dark_during_leg  personal_start95  personal_end95  \
team_id leg                                                      
3       1                0.0         34.216007       49.960080   
        2                0.0         36.464980       53.182904   
        3                0.0         38.695809       63.962235   
        4                0.0         46.775790       65.651483   
8       1                0.0         40.670099       60.345639   
        2                0.0         37.008688       59.413086   
        3                0.0         45.893713       73.668861   

             personal_median  
team_id leg                   
3       1          41.374274  
        2          44.129056  
        3          49.552665  
        4          55.521563  
8       1          49.621422  
        2          46.716714  
        3          58.219842

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist               name  \
team_id leg                                                              
3       1                          MS Parma 1   6.0  Anastasia Rudnaya   
        2                          MS Parma 1   5.7     Anni Heikkonen   
        3                          MS Parma 1   7.3     Tiia Istolahti   
        4                          MS Parma 1   7.9      Maija Sianoja   
8       1    Södertälje Nykvarn Orientering 1   6.0         Fiona Bunn   

             log_mean   log_std  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
3       1       1.928  0.096000    34.216007   49.960080   41.374274   
        2       2.046  0.098000    74.780571   97.778624   85.704413   
        3       1.916  0.128204   119.681929  154.012746  135.525621   
        4       1.950  0.086000   172.443933  212.069989  191.191155   
8       1       2.110  0.100000    40.670099   60.345639   49.621422   

               fin_mean  fin_sum_log_mean  fin_sum_log_std  mass_start  \
team_id leg                                                              
3       1     41.510883          3.721391         0.095551         0.0   
        2     85.800359          4.449700         0.068194         0.0   
        3    135.807935          4.909137         0.064870         0.0   
        4    191.517743          5.253597         0.052600         0.0   
8       1     49.856755          3.904150         0.100094         0.0   

             dark_during_leg  personal_start95  personal_end95  \
team_id leg                                                      
3       1                0.0         34.216007       49.960080   
        2                0.0         36.464980       53.182904   
        3                0.0         38.695809       63.962235   
        4                0.0         46.775790       65.651483   
8       1                0.0         40.670099       60.345639   

             personal_median               fin_time_median  \
team_id leg                                                  
3       1          41.374274 2019-06-15 14:41:22.456437111   
        2          44.129056 2019-06-15 15:25:42.264770508   
        3          49.552665 2019-06-15 16:15:31.537279367   
        4          55.521563 2019-06-15 17:11:11.469306707   
8       1          49.621422 2019-06-15 14:49:37.285306692   

                         fin_time_start95                fin_time_end95  \
team_id leg                                                               
3       1   2019-06-15 14:34:12.960390806 2019-06-15 14:49:57.604820967   
        2   2019-06-15 15:14:46.834265947 2019-06-15 15:37:46.717469692   
        3   2019-06-15 15:59:40.915768147 2019-06-15 16:34:00.764769316   
        4   2019-06-15 16:52:26.635995865 2019-06-15 17:32:04.199365854   
8       1   2019-06-15 14:40:40.205917835 2019-06-15 15:00:20.738338232   

             personal_estimate_interval  
team_id leg                              
3       1                     15.744074  
        2                     16.717925  
        3                     25.266426  
        4                     18.875694  
8       1                     19.675540

In [16]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

#for_print[for_print['team'].str.contains("Stadin")]
for_print = for_print[for_print['team'].str.contains("Reak")]

for_print = for_print[[
 'leg',
 'name', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,


In [17]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
840,341,1,Rasti-Jyry 1,Jola Maisala,2.197000,0.117000,3.988060,0.115878,2019-06-15 11:42:52.124161482+00:00,2019-06-15 12:07:54.063544273+00:00,2019-06-15 11:53:53.261002779+00:00,0.0000,0.0
841,341,2,Rasti-Jyry 1,Kaija Rantoja,2.364000,0.140000,4.745996,0.091949,2019-06-15 12:36:14.733530045+00:00,2019-06-15 13:18:21.613584995+00:00,2019-06-15 12:55:00.968222857+00:00,0.0000,0.0
842,341,3,Rasti-Jyry 1,Jenni Castren,2.395000,0.152000,5.277391,0.082499,2019-06-15 13:46:53.605127811+00:00,2019-06-15 14:49:44.627531528+00:00,2019-06-15 14:15:42.486666203+00:00,0.0000,0.0
843,341,4,Rasti-Jyry 1,Inka Lemmetyinen,2.558000,0.178309,5.700800,0.082000,2019-06-15 15:15:08.345781565+00:00,2019-06-15 16:53:11.726795197+00:00,2019-06-15 15:59:13.571730614+00:00,0.0000,0.0
1416,528,1,Rasti-Jyry 2,Päivi Taponen,2.418000,0.163000,4.211426,0.161917,2019-06-15 11:49:04.771385431+00:00,2019-06-15 12:32:34.503549099+00:00,2019-06-15 12:07:31.258761644+00:00,0.0000,0.0
1417,528,2,Rasti-Jyry 2,Miia Hovi,2.229000,0.141000,4.796115,0.110640,2019-06-15 12:37:34.788795471+00:00,2019-06-15 13:30:07.206452370+00:00,2019-06-15 13:01:09.088110447+00:00,0.0000,0.0
1418,528,3,Rasti-Jyry 2,Mervi Luurila,2.459000,0.191317,5.335030,0.102898,2019-06-15 13:49:56.700578451+00:00,2019-06-15 15:15:04.851178169+00:00,2019-06-15 14:27:17.092831373+00:00,0.0000,0.0
1419,528,4,Rasti-Jyry 2,Tiina Isoniemi,2.371000,0.116000,5.680051,0.080295,2019-06-15 15:11:16.459144831+00:00,2019-06-15 16:44:05.397117615+00:00,2019-06-15 15:52:46.434898138+00:00,0.0056,0.0
2060,740,1,Rasti-Jyry 4,Mila Viksilä,2.581289,0.144432,4.372400,0.145505,2019-06-15 11:59:40.884318829+00:00,2019-06-15 12:45:47.722437143+00:00,2019-06-15 12:19:11.973861217+00:00,0.0000,0.0
2061,740,2,Rasti-Jyry 4,Tuula Ohramaa,2.617000,0.174000,5.064254,0.113689,2019-06-15 13:06:47.486535788+00:00,2019-06-15 14:17:34.126895665+00:00,2019-06-15 13:38:17.547523499+00:00,0.0000,0.0


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [19]:
for_web.shape

(5028, 13)